In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mangaki.settings")

import django
django.setup()

from django.conf import settings

In [3]:
from mangaki.models import Rating, Work
from mangaki.utils.data import Dataset

dataset = Dataset()
#triplets = Rating.objects.values_list('user_id', 'work_id', 'choice')
#dataset.titles = dict(Work.objects.values_list('id', 'title'))
#dataset.make_anonymous_data(triplets, convert=lambda choice: 1 if choice in ['like', 'favorite'] else 0, ordered=True)
#dataset.save_csv(suffix='-ordered')
dataset.load_csv('ratings-ordered.csv', convert=float, title_filename='works-ordered.csv')

In [4]:
from mangaki.utils.efa import MangakiEFA

efa = MangakiEFA(20)
efa.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)
efa.fit(dataset.anonymized.X, dataset.anonymized.y, truncated=100)

Computing M: (2020 × 9310)
Shapes (2020, 20) (20, 100)
Chrono: factor matrix [0q, 238ms]


In [23]:
import numpy as np

np.sum(abs(np.round(efa.M + efa.model.mean_) - efa.matrix[:, :100])) / (dataset.anonymized.nb_users * 100)

0.090856435643564354

In [6]:
efa.matrix

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [35]:
fa.components_

array([[  1.27599441e-01,   1.69017079e-01,   2.03957310e-01, ...,
          1.41185926e-01,   2.80859159e-02,   1.16370846e-01],
       [  9.74821615e-02,   1.61279356e-01,   1.22312911e-01, ...,
          9.22547733e-02,   1.22905219e-04,  -6.17612977e-02],
       [  9.80778693e-03,   1.52147209e-03,   5.72452839e-02, ...,
         -6.68847267e-02,  -9.44653978e-03,   3.56717234e-02],
       ..., 
       [  1.75513722e-02,   2.04842619e-02,   1.09284209e-02, ...,
         -8.04806733e-03,  -2.81564783e-02,   5.09307735e-03],
       [ -6.60519526e-02,  -3.65372904e-02,  -2.97310570e-02, ...,
         -2.20214560e-02,   1.59269986e-03,   8.98569989e-03],
       [  4.70509568e-02,   5.11826895e-02,   5.10215482e-03, ...,
         -2.04823741e-02,  -1.31750268e-02,   9.86204080e-03]])

In [25]:
K = 20

for i in range(fa.n_components):
    ranked_titles = ['%s (%s)' % (titles[work_id], fa.components_[i][work_id]) for work_id in fa.components_[i].argsort()[::-1]]
    topK = ranked_titles[:K]
    bottomK = ranked_titles[-K:]
    print('# Component', i)
    print('=' * 20, 'TOP', '=' * 20)
    print('\n'.join(topK))
    print('=' * 20, 'BOTTOM', '=' * 20)
    print('\n'.join(bottomK))
    print()

# Component 0
==================== TOP ====================
Fate/Zero (0.276556910111)
Code Geass: Hangyaku no Lelouch (0.258603208546)
Code Geass: Hangyaku no Lelouch R2 (0.251669228991)
Fate/Zero 2nd Season (0.249291889931)
Psycho-Pass (0.24415561991)
No Game No Life (0.239617340885)
Steins;Gate (0.222104253552)
Angel Beats! (0.218943002188)
Durarara!! (0.216186353136)
Clannad (0.210615045873)
Tengen Toppa Gurren Lagann (0.208169890005)
Fullmetal Alchemist (0.207227576514)
Clannad: After Story (0.206161685304)
Kill la Kill (0.205507303331)
Fate/stay night: Unlimited Blade Works (TV) - Prologue (0.20547037088)
Toradora! (0.203999466292)
Fullmetal Alchemist: Brotherhood (0.203957310401)
Summer Wars (0.197742174974)
Fate/stay night: Unlimited Blade Works (TV) (0.197008463783)
Mon voisin Totoro (0.196025769051)
==================== BOTTOM ====================
Naruto (0.09148107501)
Black Butler (0.0909642549521)
Fairy Tail (0.0891468537075)
Rosario to Vampire (0.0720913194126)
Naruto: Sh

# Adaptive testing

In [27]:
fa.components_.T

(100, 20)